In [8]:
# CÉLULA 1: Conectar ao Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# CÉLULA 2: Carregar os dados do Google Drive (de forma otimizada)

import pandas as pd

# Caminho para o arquivo DENTRO da pasta 'mini-projeto' no seu Google Drive
caminho_do_arquivo = '/content/drive/MyDrive/mini-projeto/DENGBR23.csv'

print(f"Tentando carregar o arquivo de: {caminho_do_arquivo}")

# Lista das colunas que realmente vamos usar
colunas_para_usar = [
    'ID_MUNICIP', 'SG_UF_NOT', 'CS_SEXO',
    'DT_NOTIFIC', 'NU_IDADE_N', 'CLASSI_FIN', 'CRITERIO'
]

try:
    # Lê apenas as colunas necessárias diretamente do seu Drive
    df_dengue = pd.read_csv(
        caminho_do_arquivo,
        sep=',',
        encoding='latin-1',
        usecols=colunas_para_usar,
        on_bad_lines='skip'
    )

    print("\n✅ Arquivo carregado com sucesso do Google Drive!")
    display(df_dengue.head())

except FileNotFoundError:
    print(f"\n❌ ERRO: Arquivo não encontrado!")
    print("Verifique se o nome da pasta ('mini-projeto') e do arquivo estão corretos.")
except Exception as e:
    print(f"\n❌ Ocorreu um erro inesperado: {e}")

Tentando carregar o arquivo de: /content/drive/MyDrive/mini-projeto/DENGBR23.csv

✅ Arquivo carregado com sucesso do Google Drive!


,DT_NOTIFIC,SG_UF_NOT,ID_MUNICIP,NU_IDADE_N,CS_SEXO,CLASSI_FIN,CRITERIO
0,2023-01-01,12,120070,4017,F,10.0,2.0
1,2023-01-02,12,120005,4051,F,10.0,2.0
2,2023-01-02,12,120005,4027,F,10.0,2.0
3,2023-01-02,12,120030,4020,M,10.0,2.0
4,2023-01-02,12,120005,4032,M,10.0,2.0


In [11]:
# CÉLULA DE LIMPEZA (A QUE ESTAVA FALTANDO)

print("Iniciando a limpeza e preparação dos dados...")

try:
    # Cria uma cópia do DataFrame original para trabalhar
    df_limpo = df_dengue.copy()

    # Processa a coluna de idade 'NU_IDADE_N' para extrair a idade em anos
    df_limpo['NU_IDADE_N'] = pd.to_numeric(df_limpo['NU_IDADE_N'], errors='coerce')
    df_limpo.dropna(subset=['NU_IDADE_N'], inplace=True)
    df_limpo = df_limpo[df_limpo['NU_IDADE_N'] >= 4000]
    df_limpo['IDADE'] = df_limpo['NU_IDADE_N'] - 4000
    print("- Coluna 'IDADE' processada com sucesso.")

    print("\n✅ Processo de limpeza concluído!")
    display(df_limpo.head())

except Exception as e:
    print(f"\n❌ Ocorreu um erro inesperado durante a limpeza: {e}")

Iniciando a limpeza e preparação dos dados...
- Coluna 'IDADE' processada com sucesso.

✅ Processo de limpeza concluído!


,DT_NOTIFIC,SG_UF_NOT,ID_MUNICIP,NU_IDADE_N,CS_SEXO,CLASSI_FIN,CRITERIO,IDADE
0,2023-01-01,12,120070,4017,F,10.0,2.0,17
1,2023-01-02,12,120005,4051,F,10.0,2.0,51
2,2023-01-02,12,120005,4027,F,10.0,2.0,27
3,2023-01-02,12,120030,4020,M,10.0,2.0,20
4,2023-01-02,12,120005,4032,M,10.0,2.0,32


In [12]:
# CÉLULA 3: Análise Exploratória dos Dados

try:
    print("Iniciando a análise dos dados limpos...")

    # Pergunta 1: Quantos casos de dengue foram notificados por estado?
    print("\n--- Contagem de Casos por Estado ---")
    casos_por_estado = df_limpo['SG_UF_NOT'].value_counts()
    print(casos_por_estado)

    # Pergunta 2: Qual a distribuição de casos por sexo?
    print("\n--- Contagem de Casos por Sexo ---")
    casos_por_sexo = df_limpo['CS_SEXO'].value_counts()
    print(casos_por_sexo)

    # Pergunta 3: Quais foram os critérios de confirmação mais comuns?
    print("\n--- Contagem por Critério de Confirmação ---")
    casos_por_criterio = df_limpo['CRITERIO'].value_counts()
    print(casos_por_criterio)

    # Pergunta 4: Qual a média de idade dos pacientes notificados?
    print(f"\n--- Idade Média dos Pacientes ---")
    idade_media = df_limpo['IDADE'].mean()
    print(f"A idade média dos pacientes com dengue notificados em 2023 foi de {idade_media:.1f} anos.")

except NameError:
    print("\n❌ ERRO: A variável 'df_limpo' não foi criada. Verifique se a célula de limpeza (anterior a esta) foi executada sem erros.")
except Exception as e:
    print(f"\n❌ Ocorreu um erro inesperado durante a análise: {e}")

Iniciando a análise dos dados limpos...

--- Contagem de Casos por Estado ---
SG_UF_NOT
31    402057
35    335109
41    207860
42    141325
52     69332
33     49269
29     46623
50     44756
53     41643
43     38443
51     27819
23     14268
11     10562
26      7775
22      7510
24      7473
12      7064
25      7029
13      6126
15      5579
21      4685
27      4683
17      3239
28      2899
16      1199
14       218
Name: count, dtype: int64

--- Contagem de Casos por Sexo ---
CS_SEXO
F    803708
M    688523
I      2312
Name: count, dtype: int64

--- Contagem por Critério de Confirmação ---
CRITERIO
2.0    669005
1.0    613035
3.0     15775
Name: count, dtype: int64

--- Idade Média dos Pacientes ---
A idade média dos pacientes com dengue notificados em 2023 foi de 35.0 anos.


In [13]:
# CÉLULA 4: Visualização dos Dados

import plotly.express as px

print("--- Gerando Gráfico de Casos por Estado ---")

try:
    # Reutiliza a análise que fizemos na célula anterior para o gráfico
    casos_por_estado = df_limpo['SG_UF_NOT'].value_counts().reset_index()

    # Renomeia as colunas para o gráfico ficar mais claro
    casos_por_estado.columns = ['Estado_Codigo', 'Numero_de_Casos']

    # O código de UF (ex: 31 para MG) não é muito legível.
    # Vamos criar um "dicionário" para mapear os códigos para as siglas dos estados.
    mapa_uf = {
        11: 'RO', 12: 'AC', 13: 'AM', 14: 'RR', 15: 'PA', 16: 'AP', 17: 'TO',
        21: 'MA', 22: 'PI', 23: 'CE', 24: 'RN', 25: 'PB', 26: 'PE', 27: 'AL', 28: 'SE', 29: 'BA',
        31: 'MG', 32: 'ES', 33: 'RJ', 35: 'SP',
        41: 'PR', 42: 'SC', 43: 'RS',
        50: 'MS', 51: 'MT', 52: 'GO', 53: 'DF'
    }

    # Aplica o mapeamento para criar uma nova coluna com as siglas dos estados
    casos_por_estado['Sigla_Estado'] = casos_por_estado['Estado_Codigo'].map(mapa_uf)

    # Criando o gráfico de barras interativo
    fig = px.bar(
        casos_por_estado.sort_values('Numero_de_Casos', ascending=False), # Ordena as barras da maior para a menor
        x='Sigla_Estado',
        y='Numero_de_Casos',
        title='Número de Casos de Dengue Notificados por Estado em 2023',
        labels={'Sigla_Estado': 'Estado da Federação', 'Numero_de_Casos': 'Total de Notificações'},
        template='plotly_white'
    )

    # Mostra o gráfico no seu notebook. Você poderá passar o mouse sobre as barras para ver os valores.
    fig.show()

except NameError:
    print("\n❌ ERRO: A variável 'df_limpo' não foi criada. Verifique se a célula de limpeza e a de análise foram executadas sem erros.")
except Exception as e:
    print(f"\n❌ Ocorreu um erro inesperado durante a visualização: {e}")

--- Gerando Gráfico de Casos por Estado ---


In [14]:
# CÉLULA 5: Cria o arquivo da aplicação (app.py)

%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px

# --- CONFIGURAÇÃO DA PÁGINA ---
# Define o título que aparece na aba do navegador, o ícone e o layout da página
st.set_page_config(
    page_title="Análise de Dengue no Brasil (2023)",
    page_icon="🦟",
    layout="wide"
)

# --- FUNÇÃO DE CARREGAMENTO E CACHE DOS DADOS ---
# Esta função carrega e limpa os dados. O @st.cache_data faz com que ela rode apenas uma vez,
# tornando o app muito mais rápido quando o usuário interage com os filtros.
@st.cache_data
def carregar_dados():
    try:
        # Caminho para o arquivo DENTRO do Google Drive
        caminho_do_arquivo = '/content/drive/MyDrive/mini-projeto/DENGBR23.csv'

        # Lista das colunas que realmente vamos usar
        colunas_para_usar = [
            'ID_MUNICIP', 'SG_UF_NOT', 'CS_SEXO',
            'DT_NOTIFIC', 'NU_IDADE_N', 'CLASSI_FIN', 'CRITERIO'
        ]

        # Lê apenas as colunas necessárias diretamente do seu Drive
        df_dengue = pd.read_csv(
            caminho_do_arquivo, sep=',', encoding='latin-1',
            usecols=colunas_para_usar, on_bad_lines='skip'
        )

        # Processa a coluna de idade para extrair a idade em anos
        df_limpo = df_dengue.copy()
        df_limpo['NU_IDADE_N'] = pd.to_numeric(df_limpo['NU_IDADE_N'], errors='coerce')
        df_limpo.dropna(subset=['NU_IDADE_N'], inplace=True)
        df_limpo = df_limpo[df_limpo['NU_IDADE_N'] >= 4000]
        df_limpo['IDADE'] = df_limpo['NU_IDADE_N'] - 4000
        return df_limpo

    except FileNotFoundError:
        st.error("ERRO: Arquivo 'DENGBR23.csv' não encontrado no Google Drive. Verifique o caminho e as permissões.")
        return None
    except Exception as e:
        st.error(f"Ocorreu um erro inesperado ao carregar os dados: {e}")
        return None

# --- INTERFACE PRINCIPAL DA APLICAÇÃO ---
st.title("🦟 Dashboard de Análise de Dengue no Brasil - 2023")
st.markdown("Análise interativa dos dados de notificação de Dengue do SINAN, obtidos do Portal Brasileiro de Dados Abertos.")

# Carrega os dados usando a função que criamos
df_analise = carregar_dados()

# Só exibe o conteúdo se os dados foram carregados com sucesso
if df_analise is not None:
    st.header("Distribuição de Casos Notificados por Estado")

    casos_por_estado = df_analise['SG_UF_NOT'].value_counts().reset_index()
    casos_por_estado.columns = ['Estado_Codigo', 'Numero_de_Casos']
    mapa_uf = {
        11:'RO', 12:'AC', 13:'AM', 14:'RR', 15:'PA', 16:'AP', 17:'TO', 21:'MA', 22:'PI',
        23:'CE', 24:'RN', 25:'PB', 26:'PE', 27:'AL', 28:'SE', 29:'BA', 31:'MG', 32:'ES',
        33:'RJ', 35:'SP', 41:'PR', 42:'SC', 43:'RS', 50:'MS', 51:'MT', 52:'GO', 53:'DF'
    }
    casos_por_estado['Estado'] = casos_por_estado['Estado_Codigo'].map(mapa_uf)

    fig = px.bar(
        casos_por_estado.sort_values('Numero_de_Casos', ascending=False),
        x='Estado', y='Numero_de_Casos', title='Casos de Dengue por Estado',
        labels={'Estado': 'Estado', 'Numero_de_Casos': 'Total de Notificações'}
    )
    st.plotly_chart(fig, use_container_width=True)

    # Cria duas colunas para os gráficos menores
    st.header("Análises Adicionais")
    col1, col2 = st.columns(2)

    with col1:
        st.subheader("Distribuição por Sexo")
        casos_sexo = df_analise['CS_SEXO'].value_counts()
        st.bar_chart(casos_sexo)

    with col2:
        st.subheader("Critério de Confirmação")
        casos_criterio = df_analise['CRITERIO'].value_counts()
        st.bar_chart(casos_criterio)

    # Opção para mostrar a tabela de dados
    if st.checkbox("Mostrar tabela de dados limpos"):
        st.dataframe(df_analise)
else:
    st.warning("A aplicação não pode iniciar pois os dados não foram carregados.")

Overwriting app.py


In [16]:
# CÉLULA 6: Instala as bibliotecas e executa a aplicação

!pip install -q streamlit pyngrok

from pyngrok import ngrok
import time
import subprocess

# Coloque aqui o seu Authtoken que você já tem do site ngrok.com
AUTHTOKEN = "346zypbvE9ocMsQSJja8KsrtB4m_Q2ojQaYe6TtqBNqfRyHX"

# Configura o token e executa o dashboard
ngrok.set_auth_token(AUTHTOKEN)
ngrok.kill()
!pkill streamlit
time.sleep(5)

# Inicia o streamlit em segundo plano
process = subprocess.Popen(['streamlit', 'run', 'app.py', '--server.port=8501'])
time.sleep(10) # Damos um tempo para o servidor iniciar completamente

# Cria o link público
try:
    public_url = ngrok.connect(8501)
    print("------------------------------------------------------------------")
    print(f"🎉 SEU DASHBOARD ESTÁ PRONTO! Acesse pelo link: {public_url}")
    print("------------------------------------------------------------------")
except Exception as e:
    print(f"❌ Falha ao criar o túnel do ngrok: {e}")

------------------------------------------------------------------
🎉 SEU DASHBOARD ESTÁ PRONTO! Acesse pelo link: NgrokTunnel: "https://unpuritanical-elliptical-katalina.ngrok-free.dev" -> "http://localhost:8501"
------------------------------------------------------------------
